In [1]:
import pandas as pd
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 800)
%load_ext autotime

In [2]:
path = "/Users/metaverse/Desktop/PLS/Data/"
num_values = 18

time: 1.16 ms


In [3]:
egfr = pd.read_csv(os.path.join(path, "egfr/EGFR.csv"),sep="|")

# drop nan
egfr = egfr[(egfr.value == egfr.value)]

# get the last num_values recorded values
egfr = egfr[(~egfr.value.str.contains("0,0")) & (egfr.value.str.split(",").str.len() >= num_values)]

# if the patient has more than num_values number of egfr scores, keep the last num_value count of them
egfr.loc[(egfr.value.str.split(",").str.len() > num_values), "value"] = \
          egfr.value.str.split(",").str[-num_values:].agg(','.join)

# drop rows with non-numeric values
egfr = egfr[~egfr["value"].str.contains(r"[a-zA-Z><\-\*]")]

# drop rows where first or last value is 0
egfr = egfr[~(egfr.value.str.split(",").str[17] == "0") & ~(egfr.value.str.split(",").str[0] == "0")]

# drop all other columns, reset index
egfr = egfr[["patientid","value"]].reset_index(drop=True)

time: 44.1 s


# add BMI, Systolic, Diastolic, Weight, Cholesterol

In [4]:
# append lab values to egfr dataframe dropping patients who don't have those values
for attr in ["bmi","systolic","diastolic","cholesterol","weight"]:
    df = pd.read_csv(os.path.join(path, "original/NewData/"+attr+".csv"), index_col=0)
    sharedIDs = list(set(egfr.patientid.unique().tolist()) & set(df.patientid.unique().tolist()))
    egfr = egfr[egfr.patientid.isin(sharedIDs)]
    df = df[df.patientid.isin(sharedIDs)]
    df_sorted = df.sort_values(by='recordeddttm')
    df_sorted = df_sorted.drop_duplicates('patientid', keep='last').reset_index(drop=True)
    egfr[attr] = egfr.patientid.map(df_sorted.set_index("patientid")[attr].to_dict())

/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 8min 47s


# add demographics

In [5]:
demographics = pd.read_csv(os.path.join(path, "egfr/demo.csv"), index_col=0) 
sharedIDs = list(set(egfr.patientid.unique().tolist()) & set(demographics.patientid.unique().tolist()))
egfr = egfr[egfr.patientid.isin(sharedIDs)]
demographics = demographics[demographics.patientid.isin(sharedIDs)]

egfr["sex"] = egfr.patientid.map(demographics.set_index("patientid")["sex"].to_dict())
egfr["sex"].fillna("unknown",inplace=True)

egfr["age"] = egfr.patientid.map(demographics.set_index("patientid")["age"].to_dict())
egfr["age"].fillna(0.0,inplace=True)

/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,5,7,9,11,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 4min 3s


# Interpolate

In [6]:
# get patients that have a zero egfr value
pats_to_interp = egfr[(egfr.value.str.contains(",0,"))].patientid.unique().tolist()
inter = egfr[["patientid","value"]].copy()
inter["value"] = inter["value"].str.split(",")

# explode egfr scores
df1 = inter.value.apply(pd.Series).stack().rename('value')
df2 = df1.to_frame().reset_index(1, drop=True)
inter = df2.join(inter.patientid).reset_index(drop=True)

time: 29.1 s


In [7]:
# replace those values that are . with 0.0
inter.loc[inter.value == ".", "value"] = 0.0

# cast as float
inter["value"] = inter["value"].astype(float)

# interpolate only works on NaNs, set 0.0 to NaN
inter.loc[inter["value"] == 0.0, "value"] = float('NaN')

time: 411 ms


In [8]:
# append pats that already had a NaN in one of their values to the pats_to_interp list from above
pats_to_interp = inter[inter.value != inter.value].patientid.unique().tolist()+pats_to_interp

time: 8.64 ms


In [9]:
# interpolate missing values for only patients with missing valies
for i in pats_to_interp:
    inter[inter.patientid == i] = inter[inter.patientid == i].interpolate(method='polynomial',order=1)

time: 2min 19s


In [10]:
# make a dictionary where key is patientid and value is the interpolated egfr scores
interpo_dict = inter.groupby('patientid')['value'].apply(list).to_dict()

time: 12.6 s


In [11]:
# map the interpolated values back to the egfr dataframe
egfr["value_interped"] = egfr.patientid.map(interpo_dict)

time: 1.21 s


# One Hot Encode

In [15]:
# split scores into columns
score_cols = ["score_"+str(i) for i in range(1,num_values+1)]
temp = pd.DataFrame(egfr["value_interped"].values.tolist(), columns=score_cols, index=egfr.index)
egfr = egfr.join(temp, how='outer')

time: 514 ms


In [16]:
# encode gender
egfr = egfr[egfr.sex != "unknown"]
egfr.loc[egfr.sex == "female", "sex"] = 0
egfr.loc[egfr.sex == "male", "sex"] = 1

time: 223 ms


In [17]:
egfr.drop(["value","value_interped"], axis=1, inplace=True)

time: 24.8 ms


In [18]:
egfr[:2]

,patientid,bmi,systolic,diastolic,cholesterol,weight,sex,age,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,score_11,score_12,score_13,score_14,score_15,score_16,score_17,score_18
1,10052646804,22.83,157.0,65.0,267.0,133.0,0,88.0,29.89,25.79,29.84,25.75,24.01,27.83,27.79,23.97,23.93,27.74,34.5,29.77,29.81,34.55,25.88,22.33,27.2,22.5
2,1007831917615,29.05,120.0,72.0,145.0,180.0,1,84.0,29.00,35.00,28.00,34.00,32.00,26.00,32.00,26.00,19.00,23.00,31.0,27.00,29.00,25.00,26.00,23.00,27.0,32.0


time: 33.2 ms


# Save

In [19]:
egfr.to_csv(os.path.join(path, "original/NewData/egfr_clean.csv"))

time: 2.34 s


# Ad hoc, for BI Vis

In [4]:
import pandas as pd
path = "/Users/metaverse/Desktop/grad_school/spring_quarter/prob_stats/homework/project/spring_ps_project/egfr_bi.csv"
egfr_bi = pd.read_csv(path,index_col=0)
egfr_bi[:2]

,patientid,bmi,systolic,diastolic,cholesterol,weight,sex,age,score_1,score_2,...,score_9,score_10,score_11,score_12,score_13,score_14,score_15,score_16,score_17,score_18
1,10052646804,22.83,157.0,65.0,267.0,133.0,0,88.0,29.89,25.79,...,23.93,27.74,34.5,29.77,29.81,34.55,25.88,22.33,27.2,22.5
2,1007831917615,29.05,120.0,72.0,145.0,180.0,1,84.0,29.00,35.00,...,19.00,23.00,31.0,27.00,29.00,25.00,26.00,23.00,27.0,32.0


In [8]:
scores = ["score_"+str(i) for i in range(1,19)]

In [11]:
egfr_bi = egfr_bi[["patientid"]+scores]

In [12]:
egfr_bi[:2]

,patientid,score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10,score_11,score_12,score_13,score_14,score_15,score_16,score_17,score_18
1,10052646804,29.89,25.79,29.84,25.75,24.01,27.83,27.79,23.97,23.93,27.74,34.5,29.77,29.81,34.55,25.88,22.33,27.2,22.5
2,1007831917615,29.00,35.00,28.00,34.00,32.00,26.00,32.00,26.00,19.00,23.00,31.0,27.00,29.00,25.00,26.00,23.00,27.0,32.0


In [18]:
rename_col = {"score_"+str(i):v+1 for v,i in enumerate(range(1,19))}

In [19]:
egfr_bi.rename(columns=rename_col,inplace=True)

/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [21]:
egfr_bi.to_csv(path)